# Test Notebook

This notebook is designed to make it easier for the tester to test our model. With one click on "Run All" you can download the test sets, download the weights of the models, load them and display metrics for the two trained models.

In [13]:
import numpy as np
from keras import models
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import matplotlib.pyplot as plt
import tensorflow as tf
import shap

Download The test sets and the models

In [14]:
!gdown --id 1fe-9sU4dw6piPpoRLGDC33zYpf359w3v

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1fe-9sU4dw6piPpoRLGDC33zYpf359w3v
To: /home/dsi/ohadico97/ds_project/utils.zip
100%|██████████████████████████████████████| 48.9M/48.9M [00:01<00:00, 28.1MB/s]


Unzip the files

In [15]:
import shutil
shutil.unpack_archive("./utils.zip",'./' )

Load the test sets

In [16]:
# df_whole test samples.
X_test = np.load('./X_test.npz')
X_test0 = np.load('./X_test0.npz')
# bow_df test samples.
y_test = np.load('./y_test.npz')
y_test0 = np.load('./y_test0.npz')

Load model function

In [17]:
def load_trained_model(filename):
    model = models.load_model(f'./models/{filename}.h5')
    return model

## Load main model (df_whole)

In [18]:
classifier_whole = load_trained_model('df_whole')

2022-08-15 21:36:04.636512: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-15 21:36:07.493061: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 343 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1a:00.0, compute capability: 7.5
2022-08-15 21:36:07.494176: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 343 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1b:00.0, compute capability: 7.5
2022-08-15 21:36:07.495307: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:local

## Predicting the Test set results for main model (df_whole)

In [19]:
with tf.device('/cpu:0'):
    y_pred = classifier_whole.predict(X_test['arr_0'])
    print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test['arr_0'].reshape(len(y_test['arr_0']),1)),1))

9105/9105 [==============================] - 25s 3ms/step
[[0.02581253 0.        ]
 [0.02169383 0.        ]
 [0.14996384 0.        ]
 ...
 [0.99579245 0.        ]
 [0.00354013 0.        ]
 [0.02420975 0.        ]]


### Confusion Matrix and accuracy score and other metrics for df_whole

In [26]:
y_05_pred = (y_pred > 0.5)
cm = confusion_matrix(y_test['arr_0'], y_05_pred)
print(f'Confusion Matrix:\n\n{cm}')
accuracy = accuracy_score(y_test['arr_0'], y_05_pred)
print(f'Accuracy: {round(accuracy*100)}%')
tp, fp, fn, tn = cm.ravel()
TPR = tp/(tp+fn)
TNR = tn/(tn+fp)
print(f'TPR: {round(TPR*100)}%\n'
      f'TNR: {round(TNR*100)}%')
target_names = ['Arrested', 'Not Arrested']
print(classification_report(y_test['arr_0'], y_05_pred, target_names=target_names))

Confusion Matrix:

[[210930   4697]
 [ 27153  48563]]
Accuracy: 89%
TPR: 89%
TNR: 91%
              precision    recall  f1-score   support

    Arrested       0.89      0.98      0.93    215627
Not Arrested       0.91      0.64      0.75     75716

    accuracy                           0.89    291343
   macro avg       0.90      0.81      0.84    291343
weighted avg       0.89      0.89      0.88    291343



## Load Bag of Words model (bow_df)

In [21]:
classifier_bow = load_trained_model('bow_df')

## Predicting the Test set results for bow_df

In [22]:
with tf.device('/cpu:0'):
    y_pred0 = classifier_bow.predict(X_test0['arr_0'])
    print(np.concatenate((y_pred0.reshape(len(y_pred0),1), y_test0['arr_0'].reshape(len(y_test0['arr_0']),1)),1))

9105/9105 [==============================] - 25s 3ms/step
[[0.04361622 0.        ]
 [0.03200115 0.        ]
 [0.06523862 0.        ]
 ...
 [0.99577147 0.        ]
 [0.04611504 0.        ]
 [0.09515184 0.        ]]


### Confusion Matrix and accuracy score and other metrics for bow_df

In [23]:
y_05_pred0 = (y_pred0 > 0.5)
cm = confusion_matrix(y_test0['arr_0'], y_05_pred0)
print(f'Confusion Matrix:\n\n{cm}')
accuracy = accuracy_score(y_test0['arr_0'], y_05_pred0)
print(f'Accuracy: {round(accuracy*100)}%')
tp, fp, fn, tn = cm.ravel()
TPR = tp/(tp+fn)
TNR = tn/(tn+fp)
print(f'TPR: {round(TPR*100)}%\n'
      f'TNR: {round(TNR*100)}%')
target_names = ['Arrested', 'Not Arrested']
print(classification_report(y_test0['arr_0'], y_05_pred0, target_names=target_names))

Confusion Matrix:

[[212690   2937]
 [ 32644  43072]]
Accuracy: 88%
TPR: 87%
TNR: 94%
              precision    recall  f1-score   support

    Arrested       0.87      0.99      0.92    215627
Not Arrested       0.94      0.57      0.71     75716

    accuracy                           0.88    291343
   macro avg       0.90      0.78      0.82    291343
weighted avg       0.88      0.88      0.87    291343



## Create a combined prediction by using probabilities from both models

In [24]:
y_pred_combined = y_pred + y_pred0
y_pred_combined = y_pred_combined / 2
y_pred_combined.shape

(291343, 1)

### Confusion Matrix and accuracy score and other metrics for <b>combined</b> prediction by using probabilities from both models

In [25]:
y_05_combined = (y_pred_combined > 0.5)
cm = confusion_matrix(y_test0['arr_0'], y_05_combined)
print(f'Confusion Matrix:\n\n{cm}')
accuracy = accuracy_score(y_test0['arr_0'], y_05_combined)
print(f'Accuracy: {round(accuracy*100)}%')
tp, fp, fn, tn = cm.ravel()
TPR = tp/(tp+fn)
TNR = tn/(tn+fp)
print(f'TPR: {round(TPR*100)}%\n'
      f'TNR: {round(TNR*100)}%')
target_names = ['Arrested', 'Not Arrested']
print(classification_report(y_test0['arr_0'], y_05_combined, target_names=target_names))

Confusion Matrix:

[[211456   4171]
 [ 28047  47669]]
Accuracy: 89%
TPR: 88%
TNR: 92%
              precision    recall  f1-score   support

    Arrested       0.88      0.98      0.93    215627
Not Arrested       0.92      0.63      0.75     75716

    accuracy                           0.89    291343
   macro avg       0.90      0.81      0.84    291343
weighted avg       0.89      0.89      0.88    291343

